문제1 Poem Classification

문제1은 Poem을 전처리하여 Genre를 분류하는 task입니다. 목표 평가지수는 accuracy입니다.

(문제X) 아래 코드를 실행해 자유롭게 데이터의 분포를 확인하고, 결측치를 없애주세요

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train_df = pd.read_csv("poem_train_data.csv")
test_df = pd.read_csv("poem_test_data.csv")

In [4]:
train_df.isnull().sum()

Genre    0
Poem     4
dtype: int64

In [5]:
train_df=train_df.dropna()
train_df

,Genre,Poem
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...
5,Music,"for Bob Marley, Bavaria, November 1980 Here i..."
...,...,...
836,Environment,Why make so much of fragmentary blue In here a...
837,Environment,"Woman, I wish I didn't know your name. What co..."
838,Environment,"Yonder to the kiosk, beside the creek, Paddle ..."
839,Environment,You come to fetch me from my work to-night Whe...


문제1-1</br>
train데이터, test데이터를 concat으로 합쳐주세요

In [6]:
df = pd.concat([train_df, test_df])

In [7]:
df['Genre'].value_counts()

Environment    252
Music          250
Death          244
Affection      241
Name: Genre, dtype: int64

문제1-2</br>
df의 장르를 숫자로 인코딩 해주세요(인코딩 방식은 자유입니다.)

In [8]:
encoding = {'Music':0,"Death":1,"Environment":2,"Affection":3}
df['Genre']=df['Genre'].map(encoding)

(문제X) 아래 코드를 실행해 주세요

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

1-4 아래 전처리 함수를 완성해 주세요

In [11]:
from nltk.tokenize import word_tokenize

import re

def preprocess_text(text):
    # 토큰화 해주세요
    tokens = word_tokenize(str(text).lower())

    # 일반적인 stop_word를 정의해주세요
    stop_words = stopwords.words('english')

    #stop_word를 제거해주세요
    filtered_tokens = [word for word in tokens if word not in stop_words]

    #정규표현식으로 구두점을 정의해주세요
    regex = r"[^\w\s]"

    #이제 토큰을 받고, 구두점을 제거해주세요
    filtered_text = ' '.join(filtered_tokens)
    filtered_text = re.sub(regex, '', filtered_text)
    return filtered_text


1-5</br>
이제 위의 함수를 Poem에 적용해 Poem_Processed열에 넣어주세요
추가로 전처리 전의 문장의 길이를 Poem_len, 전처리 후의 문장의 길이를 Poem_Processed_lend에 넣어주세요

In [12]:
df['Poem_Processed'] = df['Poem'].apply(preprocess_text)
df['Poem_len'] = df['Poem'].apply(lambda x: len(str(x)))
df['Poem_Processed_len'] = df['Poem_Processed'].apply(lambda x: len(str(x)))
df.head()

,Genre,Poem,Poem_Processed,Poem_len,Poem_Processed_len
1,0,In the thick brushthey spend the...,thick brushthey spend hottest part day soakin...,205,112
2,0,Storms are generous. ...,storms generous something easy surrender sit...,191,77
3,0,—After Ana Mendieta Did you carry around the ...,after ana mendieta carry around matin star ho...,242,177
4,0,for Aja Sherrard at 20The portent may itself ...,aja sherrard 20the portent may memory wallace...,209,142
5,0,"for Bob Marley, Bavaria, November 1980 Here i...",bob marley bavaria november 1980 brilliant m...,197,137


1-6 </br>
TfidfVectorizer객체를 생성해, vectorize한뒤 train_data, train_label로 분리 해주세요

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Poem_Processed'])
train_data = X[0:len(df)]
train_label = df['Genre']


1-7 이제 train_data로 데이터를 분류해주세요(test_size =0.33, random_state = 42)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.33, random_state=42)

1-8</br>
RandomForest로 모델을 학습하고 GridSearchCV 교차검증하여 최적의 파라미터와, 최고 정확도 값을 출력해주세요(파라미터는 주어진 파라미터, cv = 5로 설정해주세요)

In [15]:
from sklearn.ensemble import  RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


rfc = RandomForestClassifier()

param_grid_rfc = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_leaf': [1, 2, 4]
}

grid_model_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid_rfc, cv=5)


grid_model_rfc.fit(X_train,y_train)
print('GridSearchCV 최적 파라미터:', grid_model_rfc.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_model_rfc.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 30, 'min_samples_leaf': 2, 'n_estimators': 100}
GridSearchCV 최고 정확도: 0.4342


2. 요약된 뉴스 내용을 이용해 카테고리 별로 분류하려고 합니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/Colab Notebooks/News_Category.json')

Mounted at /content/drive


In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

news = pd.read_json('News_Category.json', lines=True)
news.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


2-1. 'category'와 'short_description'열만 news에 저장해 주세요.


In [17]:
news = news[['category', 'short_description']]

2-2. 'category'별 빈도수를 확인하고, 가장 빈도가 높은 5개의 category에 해당하는 열만 다시 news에 저장해 주세요.

In [18]:
news['category'].value_counts()

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATION       

In [19]:
news = news[news['category'].isin(['POLITICS', 'WELLNESS', 'ENTERTAINMENT', 'TRAVEL', 'STYLE & BEAUTY'])]
news.head()

,category,short_description
20,ENTERTAINMENT,"For the past 18 months, Hollywood has effectiv..."
21,POLITICS,President issues vow as tensions with China rise.
24,POLITICS,An annual celebration took on a different feel...
28,ENTERTAINMENT,"The ""Avatar"" director said aspects of his 2009..."
30,POLITICS,"U.S. President Joe Biden, in London for the fu..."


2-3. 1-4문제에서 만든 preprocess_text 함수를 'short_description'에 적용하여 news에 'description_token'이라는 새로운 열을 추가해 주세요.

In [20]:
news['description_token'] = news['short_description'].apply(preprocess_text)
news.head()

,category,short_description,description_token
20,ENTERTAINMENT,"For the past 18 months, Hollywood has effectiv...",past 18 months hollywood effectively boycotte...
21,POLITICS,President issues vow as tensions with China rise.,president issues vow tensions china rise
24,POLITICS,An annual celebration took on a different feel...,annual celebration took different feel russia ...
28,ENTERTAINMENT,"The ""Avatar"" director said aspects of his 2009...",avatar director said aspects 2009 movie sti...
30,POLITICS,"U.S. President Joe Biden, in London for the fu...",us president joe biden london funeral queen e...


2-4. 'description_token'을 데이터, 'category'를 타겟으로 인식하여 train과 test데이터로 분리해 주세요.

In [21]:
X_train, X_test, y_train, y_test = train_test_split(news['description_token'], news['category'], test_size=0.8)

2-5. CountVectorizer을 이용해 X_train과 X_test에 대해 피처 벡터화 변환을 수행해 주세요.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)
X_test_cnt_vect = cnt_vect.transform(X_test)

2-6. 로지스틱 회귀를 적용해 예측을 수행하고 예측 정확도를 출력해 주세요.

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('예측 정확도 : {0:.3f}'.format(accuracy_score(y_test, pred)))


예측 정확도 : 0.746
